In [1]:
import os
import dotenv

dotenv.load_dotenv()

from datafin.aws import SecretsClient
from datafin.apis import PolygonClient


env_personal_aws_key = os.getenv('PERSONAL_AWS_ACCESS_KEY')
env_personal_aws_secret_key = os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY')

secrets = SecretsClient(
    aws_access_key=env_personal_aws_key,
    aws_secret_access_key=env_personal_aws_secret_key
)

In [2]:
pg = PolygonClient(secrets.get_polygon_api_key())

In [11]:
today = pg.get_aggs(
    'spy',
    1,
    'year',
    '2024-01-01',
    '2024-12-31'
)

In [12]:
today

[Agg(open=472.16, high=609.07, low=466.43, close=586.08, volume=14447832087.0, vwap=533.808, timestamp=1704085200000, transactions=118560942, otc=None)]